In [1]:
###############################
## This script links the flood sum map that was generated in GEE to the corresponding flood frequenices 
## for the Prek Area 
## exüprted from this script: https://code.earthengine.google.com/f69af6afbec2c0a5d43a173a965c67be

In [2]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
import os
import matplotlib.pyplot as plt
from io import StringIO
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import math
from matplotlib.dates import date2num
from pandas import DataFrame

In [3]:
# water level at Koh Kehl 

wl_long = pd.read_csv('wl_KK.csv')
wl_long['date'] = pd.to_datetime(wl_long['date'])
wl_long['date'] = wl_long['date'].dt.strftime('%Y-%m-%d')
wl_long.columns = ['huh', 'date', 'wl_long']
wl_long = wl_long.drop(columns=['huh'])
wl_long = wl_long.set_index('date')
wl_long['wl_long'] = wl_long['wl_long']-1 # <------ -1 correct for elevation 

# for data availability comparison:
wl_long.index = pd.to_datetime(wl_long.index)
wl_long1 = wl_long.resample('d').max()

wl_KK_daily = wl_long1  
wl_KK_daily.columns = ['wl_KK']
wl_original = wl_KK_daily

#wl_KK_daily = wl_KK_daily.rolling(center=False, window=3, min_periods=1).mean().round(2)

#wl_KK_daily.to_csv('wl_KK_smoothed.csv')

wl_KK_daily.head()

,wl_KK
date,
1990-09-05,7.16
1990-09-06,7.20
1990-09-07,7.22
1990-09-08,7.22
1990-09-09,7.21


In [4]:
#### SIMPLE: calculate fixed shifts! 
wl_KK_daily['date'] = wl_KK_daily.index
wl_KK_daily['Sentinel_date_PA'] = wl_KK_daily['date'].shift(-12)
wl_KK_daily['Sentinel_date_2C'] = wl_KK_daily['date'].shift(-26)

wl_KK_daily.head()


,wl_KK,date,Sentinel_date_PA,Sentinel_date_2C
date,,,,
1990-09-05,7.16,1990-09-05,1990-09-17,1990-10-01
1990-09-06,7.20,1990-09-06,1990-09-18,1990-10-02
1990-09-07,7.22,1990-09-07,1990-09-19,1990-10-03
1990-09-08,7.22,1990-09-08,1990-09-20,1990-10-04
1990-09-09,7.21,1990-09-09,1990-09-21,1990-10-05


In [5]:
##### import list of dates manually generated 

dates_simple = pd.read_csv('IE_PA_Simple.csv')
dates_simple['Date'] = pd.to_datetime(dates_simple['Date'])
dates_simple['Date'] = dates_simple['Date'].dt.strftime('%Y-%m-%d')
dates_simple['date'] = dates_simple['Date']

dates_simple = dates_simple.set_index('date')

df = dates_simple.drop(columns='Date')


df.tail()



""
date
2016-09-29
2017-08-14
2019-09-24
2011-10-14
2000-10-07


In [6]:
##### match them to the water levels at KK at the correct date

wl_KK_PA = wl_KK_daily

wl_KK_PA = wl_KK_PA.set_index('Sentinel_date_PA') # set the PA date to index

#wl_KK_PA = wl_KK_PA.sort_values('wl_KK') # sort by water level (ascending)

wl_KK_PA = wl_KK_PA[~wl_KK_PA.index.duplicated(keep='first')] # get rid of duplicates

flood_link = df.merge(wl_KK_PA, how='inner', left_index=True, right_index=True)

flood_link = flood_link.drop(columns=['Sentinel_date_2C'])

flood_link = flood_link.sort_values('wl_KK',ascending=False)

#flood_link['image_date'] = flood_link.index

flood_link.index = np.arange(1, len(flood_link) + 1)

flood_link['count_reached'] = flood_link.index

flood_link_original = flood_link

flood_link.head()
wl_KK_PA.head()


flood_link.head()


,wl_KK,date,count_reached
1,6.90,2000-09-25,1
2,6.88,2011-10-02,2
3,6.82,2018-09-03,3
4,6.70,2018-09-13,4
5,6.68,2019-09-12,5


In [9]:
wl_original['date'] = wl_original.index
wl_original['date'] = pd.to_datetime(wl_original['date'])
wl_original['monthday'] = wl_original['date'].dt.strftime('%m-%d')
wl_original['year'] = wl_original['date'].dt.strftime('%Y')

### define function that counts how many times in this time range, 
### a certain value was exceeded by the test date! 

def counting_times_reached(wl):
        list1 = []
        for year in years: 
            year = str(year)
            wl_year = wl_original[(wl_original['year']==year)] # filter for year 
            wl_decide = wl_year[(wl_year['monthday']<=testdate)] # filter before date
            #wl_min = wl_decide1['wl_long'].min()
            #date_min = wl_decide1[(wl_decide1.wl_long==wl_min).idxmax()]['date']  
            #wl_decide = wl_decide1[(wl_decide1['date'] > date_min)] # after dry season min
            check = (wl_decide['wl_KK'] >= wl).any()#.astype(int) # check if present
            list1.append(check)
        total_sum = float(sum(list1))/float(len(years))
        return total_sum
    

In [10]:
wl_90s = wl_original[(wl_original['year']<='2000')& (wl_original['year']>'1990')]
wl_00s = wl_original[(wl_original['year']>'2000')&(wl_original['year']<='2010')]
wl_10s = wl_original[(wl_original['year'] > '2010') & (wl_original['year']<='2020')
                    
                    ]
wl_pre08 = wl_original[(wl_original['year'] <= '2008')]
wl_post08 = wl_original[(wl_original['year'] > '2008')]

years1 = wl_90s.year.unique()
years2 = wl_00s.year.unique()
years3 = wl_10s.year.unique()

print(years1)
print(years2)
print(years3)


['1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999' '2000']
['2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009' '2010']
['2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020']


In [11]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_original.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

flood_link.to_csv('flood_link_GEE_SPA_overall')

### split into lookup tables 

### water level 

SPA_OA_wl = pd.DataFrame()

SPA_OA_wl['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_wl['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_wl['Value'] = flood_link['wl_KK'].astype(float)

SPA_OA_wl.to_excel('SPA_OA_wl.xlsx', index=False)

#### overall 

SPA_OA_OA = pd.DataFrame()

SPA_OA_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_OA_OA.to_excel('SPA_OA_OA.xlsx', index=False)


##### July 15 
SPA_OA_J15 = pd.DataFrame()

SPA_OA_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_OA_J15.to_excel('SPA_OA_J15.xlsx', index=False)


#### July 31

SPA_OA_J31 = pd.DataFrame()

SPA_OA_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_OA_J31.to_excel('SPA_OA_J31.xlsx', index=False)


#### Aug 15 
SPA_OA_A15 = pd.DataFrame()

SPA_OA_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_OA_A15.to_excel('SPA_OA_A15.xlsx', index=False)


#### Aug 31
SPA_OA_A31 = pd.DataFrame()

SPA_OA_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_OA_A31.to_excel('SPA_OA_A31.xlsx', index=False)


#### Sep 25

SPA_OA_S25 = pd.DataFrame()

SPA_OA_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_OA_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_OA_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_OA_S25.to_excel('SPA_OA_S25.xlsx', index=False)



In [12]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_90s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_90s')

### split into lookup tables 




### overall 

SPA_90s_OA = pd.DataFrame()

SPA_90s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_90s_OA.to_excel('SPA_90s_OA.xlsx', index=False)


##### July 15 
SPA_90s_J15 = pd.DataFrame()

SPA_90s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_90s_J15.to_excel('SPA_90s_J31.xlsx', index=False)


#### July 31

SPA_90s_J31 = pd.DataFrame()

SPA_90s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_90s_J31.to_excel('SPA_90s_J15.xlsx', index=False)


#### Aug 15 
SPA_90s_A15 = pd.DataFrame()

SPA_90s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_90s_A15.to_excel('SPA_90s_A15.xlsx', index=False)


#### Aug 31
SPA_90s_A31 = pd.DataFrame()

SPA_90s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_90s_A31.to_excel('SPA_90s_A31.xlsx', index=False)


#### Sep 25

SPA_90s_S25 = pd.DataFrame()

SPA_90s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_90s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_90s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_90s_S25.to_excel('SPA_90s_S25.xlsx', index=False)



In [13]:
#### implement the function for the 00s time period 1990-2021

years = wl_00s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_00s')
### split into lookup tables 



### overall 

SPA_00s_OA = pd.DataFrame()

SPA_00s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_00s_OA.to_excel('SPA_00s_OA.xlsx', index=False)


##### July 15 
SPA_00s_J15 = pd.DataFrame()

SPA_00s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_00s_J15.to_excel('SPA_00s_J15.xlsx', index=False)


#### July 31

SPA_00s_J31 = pd.DataFrame()

SPA_00s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_00s_J31.to_excel('SPA_00s_J31.xlsx', index=False)


#### Aug 15 
SPA_00s_A15 = pd.DataFrame()

SPA_00s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_00s_A15.to_excel('SPA_00s_A15.xlsx', index=False)


#### Aug 31
SPA_00s_A31 = pd.DataFrame()

SPA_00s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_00s_A31.to_excel('SPA_00s_A31.xlsx', index=False)


#### Sep 25

SPA_00s_S25 = pd.DataFrame()

SPA_00s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_00s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_00s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_00s_S25.to_excel('SPA_00s_S25.xlsx', index=False)


In [14]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_10s.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

flood_link.to_csv('flood_link_GEE_SPA_10s')

### split into lookup tables 



#### overall

SPA_10s_OA = pd.DataFrame()

SPA_10s_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_10s_OA.to_excel('SPA_10s_OA.xlsx', index=False)


##### July 15 
SPA_10s_J15 = pd.DataFrame()

SPA_10s_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_10s_J15.to_excel('SPA_10s_J15.xlsx', index=False)


#### July 31

SPA_10s_J31 = pd.DataFrame()

SPA_10s_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_10s_J31.to_excel('SPA_10s_J31.xlsx', index=False)


#### Aug 15 
SPA_10s_A15 = pd.DataFrame()

SPA_10s_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_10s_A15.to_excel('SPA_10s_A15.xlsx', index=False)


#### Aug 31
SPA_10s_A31 = pd.DataFrame()

SPA_10s_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_10s_A31.to_excel('SPA_10s_A31.xlsx', index=False)


#### Sep 25

SPA_10s_S25 = pd.DataFrame()

SPA_10s_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_10s_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_10s_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_10s_S25.to_excel('SPA_10s_S25.xlsx', index=False)



In [15]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_pre08.year.unique()

flood_link.to_csv('flood_link_GEE_SPA_pre08')

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 



### split into lookup tables 



#### overall

SPA_pre08_OA = pd.DataFrame()

SPA_pre08_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_pre08_OA.to_excel('SPA_pre08_OA.xlsx', index=False)


##### July 15 
SPA_pre08_J15 = pd.DataFrame()

SPA_pre08_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_pre08_J15.to_excel('SPA_pre08_J15.xlsx', index=False)


#### July 31

SPA_pre08_J31 = pd.DataFrame()

SPA_pre08_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_pre08_J31.to_excel('SPA_pre08_J31.xlsx', index=False)


#### Aug 15 
SPA_pre08_A15 = pd.DataFrame()

SPA_pre08_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_pre08_A15.to_excel('SPA_pre08_A15.xlsx', index=False)


#### Aug 31
SPA_pre08_A31 = pd.DataFrame()

SPA_pre08_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_pre08_A31.to_excel('SPA_pre08_A31.xlsx', index=False)


#### Sep 25

SPA_pre08_S25 = pd.DataFrame()

SPA_pre08_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_pre08_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_pre08_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_pre08_S25.to_excel('SPA_pre08_S25.xlsx', index=False)




In [16]:
#### implement the function for the OVERALL time period 1990-2021

years = wl_post08.year.unique()

testdate = '11-31'
flood_link['freq_overall'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-15'
flood_link['freq_Jul15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years

testdate = '07-31'
flood_link['freq_Jul31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-15'
flood_link['freq_Aug15'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '08-31'
flood_link['freq_Aug31'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 

testdate = '09-15'
flood_link['freq_Sep25'] = (flood_link['wl_KK'].apply(counting_times_reached))#/number_years 


flood_link.to_csv('flood_link_GEE_SPA_post08')
### split into lookup tables 



#### overall

SPA_post08_OA = pd.DataFrame()

SPA_post08_OA['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_OA['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_OA['Value'] = flood_link['freq_overall'].astype(float)

SPA_post08_OA.to_excel('SPA_post08_OA.xlsx', index=False)


##### July 15 
SPA_post08_J15 = pd.DataFrame()

SPA_post08_J15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_J15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_J15['Value'] = flood_link['freq_Jul15'].astype(float)

SPA_post08_J15.to_excel('SPA_post08_J15.xlsx', index=False)


#### July 31

SPA_post08_J31 = pd.DataFrame()

SPA_post08_J31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_J31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_J31['Value'] = flood_link['freq_Jul31'].astype(float)

SPA_post08_J31.to_excel('SPA_post08_J31.xlsx', index=False)


#### Aug 15 
SPA_post08_A15 = pd.DataFrame()

SPA_post08_A15['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_A15['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_A15['Value'] = flood_link['freq_Aug15'].astype(float)

SPA_post08_A15.to_excel('SPA_post08_A15.xlsx', index=False)


#### Aug 31
SPA_post08_A31 = pd.DataFrame()

SPA_post08_A31['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_A31['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_A31['Value'] = flood_link['freq_Aug31'].astype(float)

SPA_post08_A31.to_excel('SPA_post08_A31.xlsx', index=False)


#### Sep 25

SPA_post08_S25 = pd.DataFrame()

SPA_post08_S25['Minimum'] = (flood_link['count_reached']-0.0002).astype(float)
SPA_post08_S25['Maximum'] = (flood_link['count_reached']+0.0002).astype(float)
SPA_post08_S25['Value'] = flood_link['freq_Sep25'].astype(float)

SPA_post08_S25.to_excel('SPA_post08_S25.xlsx', index=False)





In [17]:
def average_inundation_duration(i): 
    
    ### make empty list 
    list1 = []
    
    ### get average duration for each year 
    
    for y in years: 

            #######
            # Get the wet season / dry season duration for year n & n+1 
            #######
            
            # cut to year y 
            wlnew = wl_original[(wl_original['year'] == y)] 
            
            # get minimum water level (height of dry season)
            wl_min = wlnew['wl_KK'].min() 
            
            # find the date that was reached 
            date_min = wlnew.loc[(wlnew.wl_KK==wl_min).idxmax()]['date']  # get date of miimum level
           
            # cut the series to after that date 
            wl_new_1 = wlnew[(wlnew['date']> date_min)] # cut to after the peak of the dry season 
            


            #######
            # Find the end of the dry season in the subsequent year 
            #######


            ### get the series for the subsequent year 
            
            # get subsequent year 
            year_n1 = int(y)+1
            year_n1 = str(year_n1)
            wl_yn1 = wl_original[(wl_original['year'] == year_n1)]
            
            # find minimum level that year 
            wl_min = wl_yn1['wl_KK'].min()
            
            # get date of minimum level 
            date_min = wl_yn1.loc[(wl_yn1.wl_KK==wl_min).idxmax()]['date']  
            
            # cut to before that date 
            wl_new_2 = wl_yn1[(wl_yn1['date'] <= date_min)]
            

            ## concatenate 

            wl_filter = pd.concat([wl_new_2, wl_new_1], axis=0)

            

            #######
            # threshold for the dates when the wl was above! 
            #######


            wl_above = wl_filter[(wl_filter['wl_KK'])>= i]
            
            wl_above_nr = len(wl_above.index)
            
            ## append to list 
            
            list1.append(wl_above_nr)
            

    average_duration = float(sum(list1))/float(len(years))
        
    return average_duration 

###### calculate average flood durations for all water levels for each decade 


duration_link = flood_link_original.drop(columns=['date', 'freq_overall', 'freq_Jul15', 'freq_Jul31', 'freq_Aug15', 'freq_Aug31', 'freq_Sep25'])

#duration_link = duration_link.set_index('wl_KK')

years = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
duration_link['overall'] = duration_link['wl_KK'].apply(average_inundation_duration)

years = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000']
duration_link['90s'] = duration_link['wl_KK'].apply(average_inundation_duration)

years = ['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
duration_link['00s'] = duration_link['wl_KK'].apply(average_inundation_duration)

years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
duration_link['10s'] = duration_link['wl_KK'].apply(average_inundation_duration)

years = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008']
duration_link['pre08'] = duration_link['wl_KK'].apply(average_inundation_duration)

years = ['2009', '2010','2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
duration_link['post08'] = duration_link['wl_KK'].apply(average_inundation_duration)


duration_link.head(31)

duration_link.to_excel('SPA_flood_duration.xlsx', index=False)
duration_link.to_csv('SPA_flood_duration.csv')

